In [1]:
from tensorflow.keras.models import load_model
from clean import downsample_mono, envelope
from kapre.time_frequency import STFT, Magnitude, ApplyFilterbank, MagnitudeToDecibel
from sklearn.preprocessing import LabelEncoder
import numpy as np
from glob import glob
import argparse
import os
import pandas as pd
from tqdm import tqdm

In [2]:
model = load_model("models/lstm_ultarmic.h5")
npy_paths = glob('{}/**'.format("npy_files_predict"), recursive=True)
npy_paths = sorted([x.replace(os.sep, '/') for x in npy_paths if '.npy' in x])
test_data = []
for path in npy_paths:
    test_data.append(np.load(path))
test_data = np.stack(test_data)
print(test_data.shape)
num_object = test_data.shape[0]
num_instance = test_data.shape[1]
dim_instance = test_data.shape[2:]
print("Number of Testing Object:",num_object)
print("Number of Testing instance for each object:",num_instance)
print("Dimension of each instance:",dim_instance)
test_data = test_data.reshape((*test_data.shape,1))
print(test_data.shape)

(2, 100, 36, 234)
Number of Testing Object: 2
Number of Testing instance for each object: 100
Dimension of each instance: (36, 234)
(2, 100, 36, 234, 1)


In [3]:
classes = sorted(os.listdir("npy_files_predict"))
le = LabelEncoder()
le.fit(classes)
print(classes)
labels = []
for i in range(num_object):
    for j in range(num_instance):
        labels.append(classes[i])
y_true = le.fit_transform(labels)
print(y_true)

['Drill', 'Shaver']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [24]:
y_pred = model.predict(test_data[0,:,:,:,:])

In [25]:
y_mean = np.mean(y_pred, axis=0)
print(y_mean)

[0.06509362 0.9349062 ]


In [8]:
train_data = np.abs(np.load("stft_object_clipping.npy"))
train_data = train_data.reshape((*train_data.shape,1))
print(train_data.shape)

(2, 300, 36, 234, 1)


In [19]:
y_pred = model.predict(train_data[0,:,:,:,:])
y_mean = np.mean(y_pred, axis=0)
print(y_mean)

[0.9981427  0.00185764]
